In [1]:
using Mocha
using PyPlot
using WaveletScattering
ws = WaveletScattering

Configuring Mocha...
 * CUDA       disabled by default
 * Native Ext disabled by default
Mocha configured, continue loading module...


DefaultBackend = Mocha.CPUBackend


WaveletScattering

In [19]:
spec = ws.Spec1D(log2_size = 13)
bank = ws.Bank1D(spec, max_log2_stride = 0)

data = randn(1<<13)

backend = Mocha.CPUBackend()
signal = ws.InputLayer(
    data = data,
    tops = [:signal],
    symbols = [:time])


layers = Mocha.Layer[signal]

Mocha.init(backend)

@time net = Mocha.Net("network", backend, layers);

28-Feb 17:21:58:INFO:root:Constructing net network on Mocha.CPUBackend...
28-Feb 17:21:58:INFO:root:Topological sorting 1 layers...
28-Feb 17:21:58:INFO:root:Setup layers...
28-Feb 17:21:58:INFO:root:Network constructed!
  0.003765 seconds (1.20 k allocations: 144.203 KB)


In [64]:
function (net::Mocha.Net){T,N}(data::Array{T,N})
    net.states[1].blobs[1].nodes[ws.Path()].data[:] = data
    for idlayer in 2:length(net.states)
        forward(backend, net.states[idlayer], net.states[1].blobs[idlayer-1])
    end
end

In [69]:
net(randn(1<<13))

In [71]:
net.data

LoadError: type Net has no field data